In [1]:
!pip install kaggle


In [2]:
from google.colab import files
files.upload()  # upload kaggle.json here


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nidhi09k","key":"386ed411c783d1759a7e60bc519298aa"}'}

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle datasets download -d vangap/indian-supreme-court-judgments

Dataset URL: https://www.kaggle.com/datasets/vangap/indian-supreme-court-judgments
License(s): CC-BY-NC-SA-4.0
 99% 4.74G/4.77G [01:12<00:00, 69.8MB/s]
100% 4.77G/4.77G [01:12<00:00, 70.9MB/s]


In [5]:
!unzip indian-supreme-court-judgments.zip -d /content/indian_judgments

Streaming output truncated to the last 5000 lines.
  inflating: /content/indian_judgments/pdfs/77639-1994___jonew__judis__11310.pdf  
  inflating: /content/indian_judgments/pdfs/7764-1997___jonew__judis__13268.pdf  
  inflating: /content/indian_judgments/pdfs/77640-1994___jonew__judis__11206.pdf  
  inflating: /content/indian_judgments/pdfs/77641-1994___jonew__judis__11702.pdf  
  inflating: /content/indian_judgments/pdfs/77642-1994___jonew__judis__11110.pdf  
  inflating: /content/indian_judgments/pdfs/77643-1994___jonew__judis__11608.pdf  
  inflating: /content/indian_judgments/pdfs/77644-1994___jonew__judis__11656.pdf  
  inflating: /content/indian_judgments/pdfs/77645-1994___jonew__judis__31859.pdf  
  inflating: /content/indian_judgments/pdfs/77646-1994___jonew__judis__11604.pdf  
  inflating: /content/indian_judgments/pdfs/77647-1994___jonew__judis__11395.pdf  
  inflating: /content/indian_judgments/pdfs/77648-1994___jonew__judis__11687.pdf  
  inflating: /content/indian_judgment

In [7]:
# Import required libraries
import pandas as pd
import os
import shutil
import zipfile
from google.colab import files

# Define dataset paths based on your extraction directory
csv_path = '/content/indian_judgments/judgments.csv'
pdf_dir = '/content/indian_judgments/pdfs'

# Load the judgments CSV
judgments_df = pd.read_csv(csv_path)

# Keep only English judgments (where 'language' column is NaN)
judgments_df = judgments_df[judgments_df['language'].isna()].reset_index(drop=True)

# Get list of all available PDF files
pdf_files = os.listdir(pdf_dir)

# Keep only judgments that have a corresponding PDF
judgments_df['pdf_exists'] = judgments_df['diary_no'].astype(str).apply(
    lambda x: any(x in fname for fname in pdf_files)
)
judgments_df = judgments_df[judgments_df['pdf_exists']].reset_index(drop=True)

# Randomly sample 2000 judgments for a smaller dataset
reduced_df = judgments_df.sample(n=2000, random_state=42).reset_index(drop=True)

# Create full PDF path for each record
reduced_df['pdf_path'] = reduced_df['diary_no'].astype(str).apply(
    lambda x: next((os.path.join(pdf_dir, f) for f in pdf_files if x in f), None)
)

# Save the reduced dataset to CSV
reduced_csv_path = '/content/reduced_judgments.csv'
reduced_df.to_csv(reduced_csv_path, index=False)

# Create a new folder to store reduced PDFs
reduced_pdf_dir = '/content/reduced_pdfs'
os.makedirs(reduced_pdf_dir, exist_ok=True)

# Copy the corresponding PDFs to the reduced folder
for pdf_path in reduced_df['pdf_path']:
    if pdf_path and os.path.exists(pdf_path):
        shutil.copy(pdf_path, reduced_pdf_dir)

In [10]:
# List of copied PDFs
pdf_files_reduced = os.listdir(reduced_pdf_dir)

# Define function to check if each row still has a corresponding PDF
def check_pdf_exists(row):
    diary_no = str(row['diary_no'])
    for f in pdf_files_reduced:
        if f.startswith(diary_no):
            return True
    return False

# Apply the function to verify PDF existence
reduced_df['pdf_exists'] = reduced_df.apply(check_pdf_exists, axis=1)
reduced_df = reduced_df[reduced_df['pdf_exists']].reset_index(drop=True)

# Keep only necessary columns
columns_to_keep = ['diary_no', 'case_no', 'judgment_dates', 'pdf_exists', 'pdf_path']
clean_df = reduced_df[columns_to_keep]

# Save cleaned dataset
clean_csv_path = '/content/reduced_judgements_clean.csv'
clean_df.to_csv(clean_csv_path, index=False)

# Zip reduced PDFs
zip_path = '/content/reduced_pdfs.zip'
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in os.listdir(reduced_pdf_dir):
        zipf.write(os.path.join(reduced_pdf_dir, file), file)

# Output summary
print("Clean CSV saved at:", clean_csv_path)
print("Reduced PDFs ZIP saved at:", zip_path)
print("Clean dataset shape:", clean_df.shape)

# Make downloadable
files.download(clean_csv_path)
files.download(zip_path)

Clean CSV saved at: /content/reduced_judgements_clean.csv
Reduced PDFs ZIP saved at: /content/reduced_pdfs.zip
Clean dataset shape: (1955, 5)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>